In [1]:
import pandas as pd
from tbdynamics.constants import (
    age_strata,
    organ_strata,
    compartments,

)
from tbdynamics.plotting import plot_model_vs_actual
import nevergrad as ng
from estival.wrappers import nevergrad as eng


# Import our convenience wrapper
from estival.wrappers.nevergrad import optimize_model
from tbdynamics.calib_utils import get_bcm, load_targets
from multiprocessing import cpu_count
from estival.sampling import tools as esamp
from estival.sampling.tools import SampleIterator, SampleTypes
from estival.utils.parallel import map_parallel
import pymc as pm
from estival.wrappers import pymc as epm
import arviz as az
import numpy as np

c:\Users\vbui0010\Anaconda3\envs\tbdyn\lib\site-packages\computegraph\jaxify.py:37: UserWarning: Could not import jax, model performance will be limited
  warnings.warn("Could not import jax, model performance will be limited")
WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
pd.options.plotting.backend = "plotly"

## Define Model

### Params and calibration targets

In [3]:
bcm = get_bcm()

### Running Optimization

In [4]:
# num_cores = cpu_count()
# # Set the number of workers for parallel optimization
# opt_class = ng.optimizers.TwoPointsDE
# orunner = optimize_model(bcm, opt_class=opt_class, num_workers=num_cores)
# for i in range(8):
#     rec = orunner.minimize(1000)
# mle_params = rec.value[1]
# mle_params

{'start_population_size': 2030663.5186808812,
 'contact_rate': 0.00826118413749866,
 'rr_infection_latent': 0.23884435501631182,
 'rr_infection_recovered': 0.3913387216751171,
 'progression_multiplier': 1.0373122389240734,
 'seed_time': 1891.9506347136132,
 'seed_num': 63.29879642587181,
 'seed_duration': 3.963717366422445,
 'smear_positive_death_rate': 0.44451361287370195,
 'smear_negative_death_rate': 0.034005241690413,
 'smear_positive_self_recovery': 0.1988721344231366,
 'smear_negative_self_recovery': 0.16753627680985256,
 'screening_scaleup_shape': 0.0960841165308873,
 'screening_inflection_time': 1993.8647611392325,
 'screening_end_asymp': 0.595901721871898}

In [ ]:
mle_params = {'start_population_size': 2030663.5186808812,
 'contact_rate': 0.00826118413749866,
 'rr_infection_latent': 0.23884435501631182,
 'rr_infection_recovered': 0.3913387216751171,
 'progression_multiplier': 4.0373122389240734,
 'seed_time': 1891.9506347136132,
 'seed_num': 63.29879642587181,
 'seed_duration': 3.963717366422445,
 'smear_positive_death_rate': 0.44451361287370195,
 'smear_negative_death_rate': 0.034005241690413,
 'smear_positive_self_recovery': 0.1988721344231366,
 'smear_negative_self_recovery': 0.16753627680985256,
 'screening_scaleup_shape': 0.0960841165308873,
 'screening_inflection_time': 1993.8647611392325,
 'screening_end_asymp': 0.595901721871898}

In [5]:
res = bcm.run(mle_params)
derived_df_0 = res.derived_outputs
targets = load_targets()

### Outputs

In [6]:
plot_model_vs_actual(
    derived_df_0, targets['pop'], "total_population", "Population", "Modelled vs Data"
)

In [7]:
derived_df_0[[f"total_populationXage_{i}" for i in age_strata]].plot(
    title="Modelled populatation by age group", kind="area"
)

In [8]:

plot_model_vs_actual(
    derived_df_0, targets['incidence'], "incidence", "Incidence", "Modelled vs Data"
)

In [9]:
derived_df_0[[f"prop_{compartment}" for compartment in compartments]].plot(kind="area")

In [10]:
derived_df_0[[f"total_populationXorgan_{i}" for i in organ_strata]].plot(
    title="Modelled populatation by organ status", kind="area"
)

In [11]:
plot_model_vs_actual(
    derived_df_0, targets['notifs'], "notification", "Notification", "Modelled vs Data"
)

In [12]:
plot_model_vs_actual(
    derived_df_0, targets['percentage_latent'], "percentage_latent", "Percentage latent", "Modelled vs Data"
)

In [13]:
derived_df_0['cdr'].plot()

In [14]:
plot_model_vs_actual(derived_df_0, targets['prevalence_pulmonary'], 'prevalence_pulmonary', 'Infectious prevalence', 'Modelled vs Estimation from 2017 prevalence survey')
